In [1]:
import optuna

ModuleNotFoundError: No module named 'optuna'

In [6]:
!pip list

Package                       Version
----------------------------- --------------------
absl-py                       1.4.0
alabaster                     0.7.13
albumentations                1.2.1
alembic                       1.11.1
altair                        4.2.2
anyio                         3.6.2
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
array-record                  0.2.0
arviz                         0.15.1
astropy                       5.2.2
astunparse                    1.6.3
attrs                         23.1.0
audioread                     3.0.0
autograd                      1.5
Babel                         2.12.1
backcall                      0.2.0
beautifulsoup4                4.11.2
bleach                        6.0.0
blis                          0.7.9
blosc2                        2.0.0
bokeh                         2.4.3
branca                        0.6.0
build                         0.10.0
Cach

In [3]:
import xgboost

In [4]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.3 MB/s eta 0:00:00


In [2]:
import optuna
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [3]:
df=pd.read_csv("https://raw.githubusercontent.com/sunnysavita10/svm_kernel/main/Admission_Prediction.csv")

In [4]:
df.head(3)

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337.0,118.0,4.0,4.5,4.5,9.65,1,0.92
1,2,324.0,107.0,4.0,4.0,4.5,8.87,1,0.76
2,3,NaN,104.0,3.0,3.0,3.5,8.00,1,0.72


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Serial No.         500 non-null    int64  
 1   GRE Score          485 non-null    float64
 2   TOEFL Score        490 non-null    float64
 3   University Rating  485 non-null    float64
 4   SOP                500 non-null    float64
 5   LOR                500 non-null    float64
 6   CGPA               500 non-null    float64
 7   Research           500 non-null    int64  
 8   Chance of Admit    500 non-null    float64
dtypes: float64(7), int64(2)
memory usage: 35.3 KB


In [6]:
df.shape

(500, 9)

In [7]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Serial No.,500.0,250.500000,144.481833,1.00,125.7500,250.50,375.25,500.00
GRE Score,485.0,316.558763,11.274704,290.00,308.0000,317.00,325.00,340.00
TOEFL Score,490.0,107.187755,6.112899,92.00,103.0000,107.00,112.00,120.00
University Rating,485.0,3.121649,1.146160,1.00,2.0000,3.00,4.00,5.00
SOP,500.0,3.374000,0.991004,1.00,2.5000,3.50,4.00,5.00
LOR,500.0,3.484000,0.925450,1.00,3.0000,3.50,4.00,5.00
CGPA,500.0,8.576440,0.604813,6.80,8.1275,8.56,9.04,9.92
Research,500.0,0.560000,0.496884,0.00,0.0000,1.00,1.00,1.00
Chance of Admit,500.0,0.721740,0.141140,0.34,0.6300,0.72,0.82,0.97


In [8]:
df.isnull().sum()

Serial No.            0
GRE Score            15
TOEFL Score          10
University Rating    15
SOP                   0
LOR                   0
CGPA                  0
Research              0
Chance of Admit       0
dtype: int64

In [9]:
df["GRE Score"]=df["GRE Score"].fillna(df["GRE Score"].median())
df["TOEFL Score"]=df["TOEFL Score"].fillna(df["TOEFL Score"].median())
df["University Rating"]=df["University Rating"].fillna(df["University Rating"].median())

In [10]:
df.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [11]:
X=df.drop(["Serial No.","Chance of Admit"],axis=1)
y=df["Chance of Admit"]

In [12]:
X

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,337.0,118.0,4.0,4.5,4.5,9.65,1
1,324.0,107.0,4.0,4.0,4.5,8.87,1
2,317.0,104.0,3.0,3.0,3.5,8.00,1
3,322.0,110.0,3.0,3.5,2.5,8.67,1
4,314.0,103.0,2.0,2.0,3.0,8.21,0
...,...,...,...,...,...,...,...
495,332.0,108.0,5.0,4.5,4.0,9.02,1
496,337.0,117.0,5.0,5.0,5.0,9.87,1
497,330.0,120.0,5.0,4.5,5.0,9.56,1
498,312.0,103.0,4.0,4.0,5.0,8.43,0


In [13]:
y

0      0.92
1      0.76
2      0.72
3      0.80
4      0.65
       ... 
495    0.87
496    0.96
497    0.93
498    0.73
499    0.84
Name: Chance of Admit, Length: 500, dtype: float64

In [14]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=20)

In [15]:
X_train.shape

(400, 7)

In [16]:
X_test.shape

(100, 7)

In [17]:
std_sca=StandardScaler()

In [18]:
tranformed_X_train=std_sca.fit_transform(X_train)

In [19]:
tranformed_X_train.shape

(400, 7)

In [20]:
tranformed_X_test=std_sca.transform(X_test)

In [21]:
tranformed_X_test.shape

(100, 7)

In [35]:
def objective(trail,data=X,target=y):
  train_x,test_x,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=25)
  param={
      "tree_method":"gpu_hist",
      'lambda':trail.suggest_float('lambda' , 1e-4,10.0),
      'alpha' :trail.suggest_float('alpha' , 1e-4 , 10.0),
      'colsample_bytree' :trail.suggest_categorical('colsample_bytree' , [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
      'subsample' :trail.suggest_categorical('subsample' , [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
      'learning_rate' : trail.suggest_categorical('learning_rate' , [.00001,.0003,.008,.02,.01,1,8]),
      'n_estimators' :300,
      'max_depth' :trail.suggest_categorical('max_depth', [3,4,5,6,7,8,9,10,11,12]),
      'random_state' :trail.suggest_categorical('random_state' , [10,20,30 ,2000 , 3454,243123]),
      'min_child_weight' :trail.suggest_int('min_child_weight' ,1,200)
      }
    
  Xgb_reg_model=xgb.XGBRegressor(**param)
  Xgb_reg_model.fit(train_x,y_train,eval_set=[(test_x,y_test)],verbose=True)
  y_predict=Xgb_reg_model.predict(test_x)
  mse=mean_squared_error(y_test,y_predict)
  return mse
    




In [36]:
find_param=optuna.create_study()
find_param.optimize(objective,n_trials=10)

[I 2023-05-23 16:21:43,273] A new study created in memory with name: no-name-ab519704-c69f-4b3e-b449-c221ab18b127


[0]	validation_0-rmse:1.34545
[1]	validation_0-rmse:8.99998
[2]	validation_0-rmse:61.03104
[3]	validation_0-rmse:415.73766
[4]	validation_0-rmse:2837.07032
[5]	validation_0-rmse:19312.54122
[6]	validation_0-rmse:131720.28724
[7]	validation_0-rmse:898488.60088
[8]	validation_0-rmse:6111664.74135
[9]	validation_0-rmse:41679976.64395
[10]	validation_0-rmse:284366148.91860
[11]	validation_0-rmse:1942232096.18965
[12]	validation_0-rmse:13236826048.93030
[13]	validation_0-rmse:90232553167.55170
[14]	validation_0-rmse:613776481466.27258
[15]	validation_0-rmse:4189678269125.97656
[16]	validation_0-rmse:28634157533095.65234
[17]	validation_0-rmse:195439116068467.40625
[18]	validation_0-rmse:1331969336257296.00000
[19]	validation_0-rmse:9071457366708168.00000
[20]	validation_0-rmse:61890998190962944.00000
[21]	validation_0-rmse:422471975609899008.00000
[22]	validation_0-rmse:2799784903816880640.00000
[23]	validation_0-rmse:inf
[24]	validation_0-rmse:inf
[25]	validation_0-rmse:inf
[26]	validation

[I 2023-05-23 16:21:44,411] Trial 0 finished with value: 2.4950010804720255e+73 and parameters: {'lambda': 5.578502356826906, 'alpha': 5.175242773646755, 'colsample_bytree': 0.5, 'subsample': 0.6, 'learning_rate': 8, 'max_depth': 5, 'random_state': 2000, 'min_child_weight': 83}. Best is trial 0 with value: 2.4950010804720255e+73.


[0]	validation_0-rmse:0.27220
[1]	validation_0-rmse:0.27220
[2]	validation_0-rmse:0.27220
[3]	validation_0-rmse:0.27220
[4]	validation_0-rmse:0.27220
[5]	validation_0-rmse:0.27220
[6]	validation_0-rmse:0.27220
[7]	validation_0-rmse:0.27220
[8]	validation_0-rmse:0.27220
[9]	validation_0-rmse:0.27220
[10]	validation_0-rmse:0.27220
[11]	validation_0-rmse:0.27220
[12]	validation_0-rmse:0.27220
[13]	validation_0-rmse:0.27220
[14]	validation_0-rmse:0.27220
[15]	validation_0-rmse:0.27220
[16]	validation_0-rmse:0.27220
[17]	validation_0-rmse:0.27220
[18]	validation_0-rmse:0.27220
[19]	validation_0-rmse:0.27220
[20]	validation_0-rmse:0.27220
[21]	validation_0-rmse:0.27220
[22]	validation_0-rmse:0.27220
[23]	validation_0-rmse:0.27220
[24]	validation_0-rmse:0.27220
[25]	validation_0-rmse:0.27220
[26]	validation_0-rmse:0.27220
[27]	validation_0-rmse:0.27220
[28]	validation_0-rmse:0.27220
[29]	validation_0-rmse:0.27220
[30]	validation_0-rmse:0.27220
[31]	validation_0-rmse:0.27220
[32]	validation_0-

[I 2023-05-23 16:21:45,452] Trial 1 finished with value: 0.074092 and parameters: {'lambda': 7.806598690259137, 'alpha': 8.203145679019228, 'colsample_bytree': 0.9, 'subsample': 0.3, 'learning_rate': 1e-05, 'max_depth': 6, 'random_state': 20, 'min_child_weight': 198}. Best is trial 1 with value: 0.074092.


...

[I 2023-05-23 16:21:52,625] Trial 7 finished with value: 0.074092 and parameters: {'lambda': 9.621224191824998, 'alpha': 6.35792483039497, 'colsample_bytree': 0.2, 'subsample': 0.1, 'learning_rate': 0.02, 'max_depth': 8, 'random_state': 3454, 'min_child_weight': 189}. Best is trial 5 with value: 0.004269396130779477.


[0]	validation_0-rmse:0.27220
[1]	validation_0-rmse:0.27220
[2]	validation_0-rmse:0.27220
[3]	validation_0-rmse:0.27220
[4]	validation_0-rmse:0.27220
[5]	validation_0-rmse:0.27220
[6]	validation_0-rmse:0.27220
[7]	validation_0-rmse:0.27220
[8]	validation_0-rmse:0.27220
[9]	validation_0-rmse:0.27220
[10]	validation_0-rmse:0.27220
[11]	validation_0-rmse:0.27220
[12]	validation_0-rmse:0.27220
[13]	validation_0-rmse:0.27220
[14]	validation_0-rmse:0.27220
[15]	validation_0-rmse:0.27220
[16]	validation_0-rmse:0.27220
[17]	validation_0-rmse:0.27220
[18]	validation_0-rmse:0.27220
[19]	validation_0-rmse:0.27220
[20]	validation_0-rmse:0.27220
[21]	validation_0-rmse:0.27220
[22]	validation_0-rmse:0.27220
[23]	validation_0-rmse:0.27220
[24]	validation_0-rmse:0.27220
[25]	validation_0-rmse:0.27220
[26]	validation_0-rmse:0.27220
[27]	validation_0-rmse:0.27220
[28]	validation_0-rmse:0.27220
[29]	validation_0-rmse:0.27220
[30]	validation_0-rmse:0.27220
[31]	validation_0-rmse:0.27220
[32]	validation_0-

[I 2023-05-23 16:21:53,712] Trial 8 finished with value: 0.074092 and parameters: {'lambda': 7.387809108490848, 'alpha': 9.498103794007127, 'colsample_bytree': 0.7, 'subsample': 0.1, 'learning_rate': 1e-05, 'max_depth': 3, 'random_state': 20, 'min_child_weight': 186}. Best is trial 5 with value: 0.004269396130779477.


[0]	validation_0-rmse:0.26875
[1]	validation_0-rmse:0.26549
[2]	validation_0-rmse:0.26243
[3]	validation_0-rmse:0.25969
[4]	validation_0-rmse:0.25706
[5]	validation_0-rmse:0.25401
[6]	validation_0-rmse:0.25121
[7]	validation_0-rmse:0.24830
[8]	validation_0-rmse:0.24563
[9]	validation_0-rmse:0.24299
[10]	validation_0-rmse:0.24026
[11]	validation_0-rmse:0.23746
[12]	validation_0-rmse:0.23485
[13]	validation_0-rmse:0.23235
[14]	validation_0-rmse:0.22999
[15]	validation_0-rmse:0.22748
[16]	validation_0-rmse:0.22497
[17]	validation_0-rmse:0.22254
[18]	validation_0-rmse:0.22026
[19]	validation_0-rmse:0.21799
[20]	validation_0-rmse:0.21581
[21]	validation_0-rmse:0.21393
[22]	validation_0-rmse:0.21175
[23]	validation_0-rmse:0.20957
[24]	validation_0-rmse:0.20766
[25]	validation_0-rmse:0.20561
[26]	validation_0-rmse:0.20390
[27]	validation_0-rmse:0.20219
[28]	validation_0-rmse:0.20024
[29]	validation_0-rmse:0.19844
[30]	validation_0-rmse:0.19648
[31]	validation_0-rmse:0.19498
[32]	validation_0-

[I 2023-05-23 16:21:54,943] Trial 9 finished with value: 0.01066138707518386 and parameters: {'lambda': 2.409993443754128, 'alpha': 9.885268527233153, 'colsample_bytree': 1, 'subsample': 0.6, 'learning_rate': 0.02, 'max_depth': 12, 'random_state': 3454, 'min_child_weight': 41}. Best is trial 5 with value: 0.004269396130779477.


In [37]:
find_param.best_trial.params

{'lambda': 7.68332415528326,
 'alpha': 0.3507735487420109,
 'colsample_bytree': 1,
 'subsample': 0.5,
 'learning_rate': 1,
 'max_depth': 11,
 'random_state': 243123,
 'min_child_weight': 39}

In [25]:
find_param.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_alpha,params_colsample_bytree,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_random_state,params_subsample,state
0,0,0.010343,2023-05-23 16:19:33.209111,2023-05-23 16:19:33.969879,0 days 00:00:00.760768,8.711156,0.7,2.302643,1.00000,9,50,243123,0.5,COMPLETE
1,1,0.074066,2023-05-23 16:19:33.973318,2023-05-23 16:19:34.103854,0 days 00:00:00.130536,8.101601,0.3,2.935958,0.00001,11,156,20,0.8,COMPLETE
2,2,0.029185,2023-05-23 16:19:34.107509,2023-05-23 16:19:34.254358,0 days 00:00:00.146849,1.175581,0.6,1.507970,0.02000,8,98,20,0.8,COMPLETE
3,3,0.055932,2023-05-23 16:19:34.256530,2023-05-23 16:19:34.407632,0 days 00:00:00.151102,2.833042,0.3,0.568995,0.00800,10,138,243123,0.5,COMPLETE
4,4,0.074092,2023-05-23 16:19:34.411603,2023-05-23 16:19:34.562745,0 days 00:00:00.151142,9.570961,0.1,0.934212,0.01000,9,188,30,0.3,COMPLETE
5,5,0.040080,2023-05-23 16:19:34.565250,2023-05-23 16:19:34.712971,0 days 00:00:00.147721,2.000214,0.8,3.721794,0.02000,10,158,20,0.5,COMPLETE
6,6,0.054609,2023-05-23 16:19:34.715693,2023-05-23 16:19:34.882425,0 days 00:00:00.166732,8.218111,1.0,3.052257,0.01000,6,38,30,0.4,COMPLETE
7,7,0.040989,2023-05-23 16:19:34.888648,2023-05-23 16:19:35.036929,0 days 00:00:00.148281,3.607215,0.2,2.395934,0.02000,9,176,3454,0.5,COMPLETE
8,8,0.012367,2023-05-23 16:19:35.038944,2023-05-23 16:19:35.215502,0 days 00:00:00.176558,8.526989,0.6,2.071582,1.00000,6,33,10,0.6,COMPLETE
9,9,0.073457,2023-05-23 16:19:35.223041,2023-05-23 16:19:35.391350,0 days 00:00:00.168309,9.775592,0.5,2.442963,0.01000,3,35,2000,0.1,COMPLETE


In [26]:
optuna.visualization.plot_optimization_history(find_param)

In [53]:
best_params={'lambda': 7.68332415528326,
 'alpha': 0.3507735487420109,
 'colsample_bytree': 1,
 'subsample': 0.5,
 'learning_rate': 1,
 'max_depth': 11,
 'random_state': 243123,
 'min_child_weight': 39}


In [55]:
model=xgb.XGBRegressor(**best_params)

In [56]:
model.fit(X_train,y_train)

XGBRegressor(alpha=0.3507735487420109, base_score=None, booster=None,
             callbacks=None, colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=1, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, lambda=7.68332415528326,
             learning_rate=1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=11,
             max_leaves=None, min_child_weight=39, missing=nan,
             monotone_constraints=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, ...)

In [57]:
y_pred=model.predict(X_test)

In [58]:
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

0.7633063646952729

In [34]:
from sklearn.ensemble import RandomForestRegressor
model2=RandomForestRegressor()
model2.fit(X_train,y_train)
y_pred2=model2.predict(X_test)
r2_score(y_test,y_pred2)


0.7958799427165991

In [ ]:
1. YOU NEED TO TAKE ONE REGRESSION DATASET AND ONE CLASSIFICAITON DATASET
2. NOW TEST WITH EACH AND EVERY ENSEMBLE TECHNIQUE ALONG WITH HYPERPARAMETER TUNING AND COMPARE THE ACCURACIES.
3. FOR THE HP TUNING USE GRRID SEARCH AND RANDOM SREACH AND OPTUNA AND HYPEROPT FOR HYPERPARAMETER TUNIUG
4. WRITE 5 DIFFERENCE BETWEEN ALL THE LIBRARY(GRID, RANDOM, HYPEROPT, OPTUNA)

CLASSIFICAITON DATASET: https://www.kaggle.com/datasets/ineubytes/heart-disease-dataset
REGRESSION DATASET: https://www.kaggle.com/datasets/arnabchaki/data-science-salaries-2023

YOU CAN SEND ME HERE: sunny.savita@ineuron.ai


